We'll pull from two apis to get flood and weather data

[Global Flood API](https://open-meteo.com/en/docs/flood-api)
[Historical Weather API](https://open-meteo.com/en/docs/historical-weather-api)

In [ ]:
# dependencies
import pandas as pd
import requests
from urllib import parse

In [46]:

def get_openmeteo_data(lat: int, lon: int, start: str, end: str, api: str):
  '''
  Obtain daily data from the api for a supplied location
  `api` can be 'flood', 'weather'
  '''

  if api == 'weather':
    base_url = 'https://archive-api.open-meteo.com/v1/archive?timezone=auto&'

    fields = [
      'temperature_2m_max',
      'temperature_2m_min',
      'temperature_2m_mean',
      'precipitation_sum',
      'precipitation_hours',
      'windspeed_10m_max'
      ]
    
  elif api == 'flood':
    base_url = 'https://flood-api.open-meteo.com/v1/flood?'

    fields = [
      'river_discharge'
      ]
    
  else:
    raise Exception("api must be: `flood` or `weather`")  

  request_dict = {
    'latitude': lat,
    'longitude': lon,
    'start_date': start,
    'end_date': end
  }

  query_url = (
    base_url
    + parse.urlencode(request_dict)
    + "&daily="
    + ','.join(fields)
  )
  
  try:
    response = requests.get(query_url)
  except:
    print("API call failed")

  df = pd.DataFrame(response.json()['daily'])
  
  dfo = df.rename(columns={'time': 'date'})
  dfo.insert(1,'latitude',lat)
  dfo.insert(2,'longitude',lon)

  return dfo

# example
get_openmeteo_data(52,13,'2023-06-24','2023-06-26','weather')

,date,latitude,longitude,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,precipitation_hours,windspeed_10m_max
0,2023-06-24,52,13,24.0,14.6,19.2,0.1,1.0,14.3
1,2023-06-25,52,13,25.5,16.3,21.8,0.3,3.0,6.9
2,2023-06-26,52,13,29.0,17.4,22.6,1.8,3.0,20.1


In [47]:
def flood_and_weather_data(lat: int, lon: int, start: str, end: str):
  '''Fetch from both endpoints and combine'''

  weather_df = get_openmeteo_data(lat,lon,start,end,'weather')
  flood_df = get_openmeteo_data(lat,lon,start,end,'flood')
  joined_df = pd.merge(weather_df,flood_df,on=['date', 'latitude', 'longitude'])
  return joined_df

# example  
test_df = flood_and_weather_data(52,13,'2023-06-24','2023-06-26')
test_df

,date,latitude,longitude,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,precipitation_hours,windspeed_10m_max,river_discharge
0,2023-06-24,52,13,24.0,14.6,19.2,0.1,1.0,14.3,0.31
1,2023-06-25,52,13,25.5,16.3,21.8,0.3,3.0,6.9,0.31
2,2023-06-26,52,13,29.0,17.4,22.6,1.8,3.0,20.1,0.44


In [64]:
# download all data and save as csv

locations = {
  'Brisbane': [-27.5,153],
  'Townsville': [-19.659,147.412]
}

dates = ['1990-01-01','2021-01-01']

combined_df = pd.DataFrame()

for location in locations:
  location_data = flood_and_weather_data(*locations[location], *dates)
  location_data.insert(1,'location',str(location))
  combined_df = pd.concat([combined_df, location_data], ignore_index=True)
  print("Data retrieved for "+location)

combined_df.to_csv('Data/flooddata.csv', index=False)

combined_df

Data retrieved for Brisbane
Data retrieved for Townsville


,date,location,latitude,longitude,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,precipitation_hours,windspeed_10m_max,river_discharge
0,1990-01-01,Brisbane,-27.500,153.000,28.6,18.3,22.5,0.0,0.0,14.1,33.20
1,1990-01-02,Brisbane,-27.500,153.000,29.3,17.0,22.9,0.0,0.0,13.4,31.58
2,1990-01-03,Brisbane,-27.500,153.000,30.8,19.3,24.1,0.0,0.0,13.1,30.41
3,1990-01-04,Brisbane,-27.500,153.000,35.5,18.5,26.0,0.0,0.0,16.8,29.48
4,1990-01-05,Brisbane,-27.500,153.000,27.4,22.0,24.3,0.0,0.0,19.4,28.85
...,...,...,...,...,...,...,...,...,...,...,...
22643,2020-12-28,Townsville,-19.659,147.412,28.5,24.6,26.4,3.6,13.0,20.1,26.54
22644,2020-12-29,Townsville,-19.659,147.412,25.8,23.9,24.6,25.1,23.0,13.6,26.99
22645,2020-12-30,Townsville,-19.659,147.412,26.1,24.0,24.8,32.8,24.0,8.9,28.31
22646,2020-12-31,Townsville,-19.659,147.412,27.2,22.9,24.5,59.4,22.0,12.5,31.58
